In [ ]:
import settings
from pyzotero import zotero

zot = zotero.Zotero(settings.USER_ID, 'user', settings.API_KEY)

import zotscripts as zs

## Collections

In [ ]:
all_colls = zot.collections()

### Automatically tag items with collection name, for all subcollections of  `zotscripts_tag`

In [ ]:
zotscripts_tag = zs.coll_key_from_name('zotscripts_tag', all_colls)
print(zotscripts_tag)

In [ ]:
zs.autotag_subcoll_items(zot, zotscripts_tag)

### One-off script to add tags to items in specific collections according to a dict

In [ ]:
# Define a dict of tags to be added to each specific collection,
# by collection name.
coll_tags = {
    'Bioinformatics': ['Bioinformatics'],
    'Biomonitoring': ['Biomonitoring', 'Exposure'],
    'Body burden': ['Body Burden'],
    'Cheminformatics': ['Cheminformatics'],
    'Computational': ['Computational Toxicology'],
    'Exposure': ['Exposure'],
    'Nanomaterials': ['Nanomaterials'],
    'Nanotoxicology': ['Nanomaterials', 'Nanotoxicology'],
}

# Get a dict of name: key for all collections.
names_keys = zs.coll_names_keys(all_colls)

# Match collection keys with lists of tags.
keys_tags = {names_keys[name]: coll_tags[name] for name in coll_tags}

# Add tags to items...
for key, tags in keys_tags.items():
    for item in zot.collection_items(key):
        zot.add_tags(item, *tags)
        try:
            print('Adding tags', tags, 'to', item['data']['title'])
        except KeyError:
            pass

## Titles

### Dry run of sentence-casing titles for all items in a collection

In [ ]:
coll_id = settings.TEST_COLL_ID

print(zot.num_collectionitems(coll_id), 'items in collection.')
items = zs.all_coll_items(zot, coll_id)

for item in items:
    new_item = zs.item_titles_to_sentence(item)
    # No need to print anything extra
    # No updating

### One-off script to update ONLY `shortTitle` and `bookTitle`

In [ ]:
# from pyzotero.zotero_errors import PreConditionFailed

coll_id = settings.TEST_COLL_ID

print(zot.num_collectionitems(coll_id), 'items in collection.')
items = zs.all_coll_items(zot, coll_id)

for item in items:
    new_item = zs.item_titles_to_sentence(item, keys=['shortTitle', 'bookTitle'])
#     if new_item:
#         try:
#             zot.update_item(new_item)
#         except PreConditionFailed:
#             print('{:<10}'.format('FAILED'), item['data']['title'])

## Journal article metadata

### One-off script to clear  `accessDate` from all journal articles in a collection

In [ ]:
coll_id = settings.TEST_COLL_ID

print(zot.num_collectionitems(coll_id), 'items in collection.')
items = zs.all_coll_items(zot, coll_id)

for item in items:
    if item['data']['itemType'] != 'journalArticle':
        continue
    
    if item['data']['accessDate']:
        item['data']['accessDate'] = ''
        print('Cleared accessDate:', item['data']['key'])
        zot.update_item(item)